# Computer vision - Week_08 - Morphological operations

In [2]:
import numpy as np
from skimage import data, filters, draw
import matplotlib.pyplot as plt

from skimage.morphology import disk
plt.rcParams['text.color'] = 'black'
plt.rcParams['xtick.color'] = 'black'
plt.rcParams['ytick.color'] = 'black'
plt.rcParams['axes.labelcolor'] = 'black'
plt.rcParams['font.size'] = 18